In [ ]:
%matplotlib inline


# Compute coherence in source space using a MNE inverse solution


This examples computes the coherence between a seed in the left
auditory cortex and the rest of the brain based on single-trial
MNE-dSPM inverse solutions.




In [ ]:
#NOT USED AS OF NOW!!!!!!!!!!



# Author: Martin Luessi <mluessi@nmr.mgh.harvard.edu>
#
# License: BSD (3-clause)

import numpy as np

import mne
from mne.datasets import sample
from mne.minimum_norm import (apply_inverse, apply_inverse_epochs,
                              read_inverse_operator)
from mne.connectivity import seed_target_indices, spectral_connectivity

print(__doc__)

data_path = sample.data_path()
subjects_dir = data_path + '/subjects'
fname_inv = data_path + '/MEG/sample/sample_audvis-meg-oct-6-meg-inv.fif'
fname_raw = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
fname_event = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw-eve.fif'
label_name_lh = 'Aud-lh'
fname_label_lh = data_path + '/MEG/sample/labels/%s.label' % label_name_lh

event_id, tmin, tmax = 1, -0.2, 0.5
method = "dSPM"  # use dSPM method (could also be MNE or sLORETA)

# Load data.
inverse_operator = read_inverse_operator(fname_inv)
label_lh = mne.read_label(fname_label_lh)
raw = mne.io.read_raw_fif(fname_raw)
events = mne.read_events(fname_event)

# Add a bad channel.
raw.info['bads'] += ['MEG 2443']

# pick MEG channels.
picks = mne.pick_types(raw.info, meg=True, eeg=False, stim=False, eog=True,
                       exclude='bads')

# Read epochs.
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks,
                    baseline=(None, 0),
                    reject=dict(mag=4e-12, grad=4000e-13, eog=150e-6))

# First, we find the most active vertex in the left auditory cortex, which
# we will later use as seed for the connectivity computation.
snr = 3.0
lambda2 = 1.0 / snr ** 2
evoked = epochs.average()
stc = apply_inverse(evoked, inverse_operator, lambda2, method,
                    pick_ori="normal")

# Restrict the source estimate to the label in the left auditory cortex.
stc_label = stc.in_label(label_lh)

# Find number and index of vertex with most power.
src_pow = np.sum(stc_label.data ** 2, axis=1)
seed_vertno = stc_label.vertices[0][np.argmax(src_pow)]
seed_idx = np.searchsorted(stc.vertices[0], seed_vertno)  # index in orig stc

# Generate index parameter for seed-based connectivity analysis.
n_sources = stc.data.shape[0]
indices = seed_target_indices([seed_idx], np.arange(n_sources))

# Compute inverse solution and for each epoch. By using "return_generator=True"
# stcs will be a generator object instead of a list. This allows us so to
# compute the coherence without having to keep all source estimates in memory.

snr = 1.0  # use lower SNR for single epochs
lambda2 = 1.0 / snr ** 2
stcs = apply_inverse_epochs(epochs, inverse_operator, lambda2, method,
                            pick_ori="normal", return_generator=True)

# Now we are ready to compute the coherence in the alpha and beta band.
# fmin and fmax specify the lower and upper freq. for each band, resp.
fmin = (8., 13.)
fmax = (13., 30.)
sfreq = raw.info['sfreq']  # the sampling frequency

# Now we compute connectivity. To speed things up, we use 2 parallel jobs
# and use mode='fourier', which uses a FFT with a Hanning window
# to compute the spectra (instead of multitaper estimation, which has a
# lower variance but is slower). By using faverage=True, we directly
# average the coherence in the alpha and beta band, i.e., we will only
# get 2 frequency bins.
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(
    stcs, method='coh', mode='fourier', indices=indices,
    sfreq=sfreq, fmin=fmin, fmax=fmax, faverage=True, n_jobs=1)

print('Frequencies in Hz over which coherence was averaged for alpha: ')
print(freqs[0])
print('Frequencies in Hz over which coherence was averaged for beta: ')
print(freqs[1])

# Generate a SourceEstimate with the coherence. This is simple since we
# used a single seed. For more than one seeds we would have to split coh.
# Note: We use a hack to save the frequency axis as time.
tmin = np.mean(freqs[0])
tstep = np.mean(freqs[1]) - tmin
coh_stc = mne.SourceEstimate(coh, vertices=stc.vertices, tmin=1e-3 * tmin,
                             tstep=1e-3 * tstep, subject='sample')

# Now we can visualize the coherence using the plot method.
brain = coh_stc.plot('sample', 'inflated', 'both',
                     time_label='Coherence %0.1f Hz',
                     subjects_dir=subjects_dir,
                     clim=dict(kind='value', lims=(0.25, 0.4, 0.65)))
brain.show_view('lateral')

In [5]:
#NOT USED AS OF NOW!!!!!!!!!!

import mne  
raw = mne.io.read_raw_edf('Testdata.bdf')  # load data  
#raw.info['bads'] = ['MEG 2443', 'EEG 053']  # mark bad channels  
raw.load_data(raw)
raw.filter(l_freq=None, h_freq=40.0)  # low-pass filter  
#events = mne.find_events(raw, 'STI014')  # extract events and epoch data 
#epochs = mne.Epochs(raw, events, event_id=1, tmin=-0.2, tmax=0.5,  reject=dict(grad=4000e-13, mag=4e-12, eog=150e-6))  
evoked = epochs.average()  # compute evoked  
evoked.plot()  # butterfly plot the evoked data 
cov = mne.compute_covariance(epochs, tmax=0, method='shrunk')  
fwd = mne.read_forward_solution(fwd_fname, surf_ori=True)  
inv = mne.minimum_norm.make_inverse_operator(  
    raw.info, fwd, cov, loose=0.2)  # compute inverse operator 
stc = mne.minimum_norm.apply_inverse(  
    evoked, inv, lambda2=1. / 9., method='dSPM')  # apply it 
stc_fs = stc.morph('fsaverage')  # morph to fsaverage 
stc_fs.plot()  # plot source data on fsaverage's brain 

Extracting edf Parameters from Testdata.bdf...
Setting channel info structure...
Creating Raw.info structure...
Ready.


TypeError: Level not an integer or a valid string: <RawEDF  |  Testdata.bdf, n_channels x n_times : 65 x 1140736 (557.0 sec), ~135 kB, data not loaded>

In [44]:
import mne
from mne.minimum_norm import (apply_inverse, apply_inverse_epochs,
                              read_inverse_operator)
from mne.connectivity import seed_target_indices, spectral_connectivity
from mne.preprocessing import run_ica

raw = mne.io.read_raw_eeglab('Exp1_annotated.set')  # load data  
events=mne.find_events(raw)
#events = mne.read_events('Experiment 1_Tu18_7_11_12_annotations.txt')
#epochs = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks,baseline=(None, 0), reject=dict(mag=4e-12, grad=4000e-13, eog=150e-6))

ica=run_ica(raw, n_components=1, max_pca_components=78, n_pca_components=64, noise_cov=None, 
    random_state=None, picks=None, start=None, stop=None, start_find=None, stop_find=None, 
    ecg_ch=None, ecg_criterion=0.1, eog_ch=None, 
    eog_criterion=0.1, skew_criterion=-1, kurt_criterion=-1, var_criterion=0, add_nodes=None, verbose=None)



Reading Exp1_annotated.fdt
38 events found
Events id: [   7    8    9   15  131  132  133  134  136  137  139  960  991 1310 1311
 9100 9115]
Fitting ICA to data using 78 channels. 
Please be patient, this may take some time
Selection by number: 1 components
<ICA  |  raw data decomposition, fit (fastica): 4814848 samples, 1 components, channels used: "eeg">
    Now searching for artifacts...
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    0, 0, 0
    Removing duplicate indices...
Ready.


In [47]:
from mne.preprocessing import ICA

ica.save("exp1-ica.fif")

Writing ica solution to exp1-ica.fif...


<ICA  |  raw data decomposition, fit (fastica): 4814848 samples, 1 components, channels used: "eeg", 1 sources marked for exclusion>

In [57]:

#rest epochs
eog=mne.Epochs(raw, events, 7, 0, 20, baseline=(None, 0))
emg=mne.Epochs(raw, events, 8, 0, 20, baseline=(None, 0))
noise=mne.epochs.concatenate_epochs([eog,emg])
rest = mne.Epochs(raw, events, 9, 0, 15, baseline=(None, 0))
rest1= mne.Epochs(raw, events, 960, 0, 60, baseline=(None, 0))
rest2= mne.Epochs(raw, events, 9115, 0, 115, baseline=(None, 0))
rest3= mne.Epochs(raw, events, 9100, 0, 100, baseline=(None, 0))
rest4= mne.Epochs(raw, events, 991, 0, 91, baseline=(None, 0))
r=[rest,rest1,rest2,rest3,rest4]
stim1 = mne.Epochs(raw, events, 131, 0, 144, baseline=(None, 0))
stim2 = mne.Epochs(raw, events, 132, 0, 103, baseline=(None, 0))
stim3 = mne.Epochs(raw, events, 133, 0, 119, baseline=(None, 0))
stim4 = mne.Epochs(raw, events, 134, 0, 171, baseline=(None, 0))

stim5 = mne.Epochs(raw, events, 136, 0, 94, baseline=(None, 0))
stim6 = mne.Epochs(raw, events, 137, 0, 279, baseline=(None, 0))

stim7 = mne.Epochs(raw, events, 139, 0, 180, baseline=(None, 0))
stim8 = mne.Epochs(raw, events, 1310, 0, 107, baseline=(None, 0))
stim9 = mne.Epochs(raw, events, 1311, 0, 286, baseline=(None, 0))
s=[stim1,stim2,stim3,stim4,stim5,stim6,stim7,stim8,stim9]

1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
Loading data for 1 events and 40961 original time points ...
0 bad epochs dropped
Loading data for 1 events and 40961 original time points ...
0 bad epochs dropped
2 matching events found
0 bad epochs dropped
8 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activated
1 matching events found
0 projection items activa

In [67]:
#evoked = epochs.average()  # compute evoked  
#evoked.plot()  # butterfly plot the evoked data 
#cov = mne.compute_covariance(epochs, tmax=0, method='shrunk')  
#ica = run_ica(epochs)

"""ica=run_ica(epochs, n_components=1, max_pca_components=50, n_pca_components=64, noise_cov=None, 
    random_state=None, picks=None, start=None, stop=None, start_find=None, stop_find=None, 
    ecg_ch=None, ecg_criterion=0.1, eog_ch=None, 
    eog_criterion=0.1, skew_criterion=-1, kurt_criterion=-1, var_criterion=0, add_nodes=None, verbose=None)

"""
coh_dump=[]
freqs_dump=[]

#snr = 3.0
#lambda2 = 1.0 / snr ** 2
#evoked = epochs.average()
#stc = apply_inverse(evoked, inverse_operator, lambda2, "dSPM",pick_ori="normal")
    
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(rest1, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(rest2, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(rest3, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(rest4, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(rest, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh_rest=coh_dump
freqs_rest=freqs_dump
coh_dump=[]
freqs_rest=[]

coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(stim1, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(stim2, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(stim3, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(stim4, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(stim5, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(stim6, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(stim7, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(stim8, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh, freqs, times, n_epochs, n_tapers = spectral_connectivity(stim9, method='wpli', mode='fourier', faverage=True, n_jobs=1)
coh_dump.extend(coh)
freqs_dump.extend(coh)
coh_stim=coh_dump
freqs_stim=freqs_dump


Connectivity computation...
    computing connectivity for 3081 connections
    using t=0.000s..60.000s for estimation (122881 points)
    frequencies: 0.1Hz..1024.0Hz (61436 points)
    connectivity scores will be averaged for each band
    using FFT with a Hanning window to estimate spectra
    the following metrics will be computed: WPLI
    computing connectivity for epoch 1
    assembling connectivity matrix
[Connectivity computation done]
Connectivity computation...
    computing connectivity for 3081 connections
    using t=0.000s..115.000s for estimation (235521 points)
    frequencies: 0.0Hz..1024.0Hz (117756 points)
    connectivity scores will be averaged for each band
    using FFT with a Hanning window to estimate spectra
    the following metrics will be computed: WPLI
    computing connectivity for epoch 1


MemoryError: 